In [2]:
import requests
from bs4 import BeautifulSoup
import os
import csv

In [30]:
def extract_cars(url,main_div,estructura_1,estructura_2,imagen):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # clase_sup = soup.find_all("lo", {"class":'ui-search-layout ui-search-layout--grid'})
    divs = soup.find_all(main_div[0], {"class":main_div[1]})
    # print(divs)
    carros = []
    for item in divs:
        dict_list = {}
        clase_alta = BeautifulSoup(str(item),'html.parser')
        nombre_carro = clase_alta.find(estructura_1[0], class_=estructura_1[1]).text
        precio_carro = clase_alta.find(estructura_2[0], class_=estructura_2[1]).text
        imagenes = clase_alta.find_all(imagen[0], class_=imagen[1])
        for img in imagenes:
            url_imagen = img.get('src')
            if not url_imagen.endswith('.webp'):
                url_imagen = img.get('data-src')
            else:
                url_imagen = img.get('src')
        dict_list['Nombre'] = nombre_carro
        dict_list['Precio'] = precio_carro
        dict_list['Imagen_URL'] = url_imagen
        carros.append(dict_list)
    return carros

In [37]:
url = 'https://carros.tucarro.com.co/_Desde_529_NoIndex_True'
main_div = ['div','andes-card poly-card poly-card--grid-card andes-card--flat andes-card--padding-0 andes-card--animated']
estructura_1 = ['h2','poly-box poly-component__title']
estructura_2 = ['span','andes-money-amount__fraction']
imagen = ['img','poly-component__picture']
carros = extract_cars(url,main_div,estructura_1,estructura_2,imagen)

In [38]:
carros
with open('carros.csv', mode='a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['Nombre_Imagen', 'Precio'])
    file.seek(0, os.SEEK_END)  
    if file.tell() == 0:
        writer.writerow(['Nombre_Imagen', 'Precio']) 
    for item in carros:
        try:
            nombre_imagen = item['Nombre'].replace(' ', '_') + '.webp'
            ruta_imagen = os.path.join('imagenes_carros', nombre_imagen)
            response = requests.get(item['Imagen_URL'])
            response.raise_for_status()
            with open(ruta_imagen, 'wb') as f:
                f.write(response.content)
            writer.writerow([nombre_imagen, item['Precio']])
        except Exception as e:
            continue

In [53]:
def extract_cars_2(url,main_div,estructura_1,estructura_2,img_src):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # clase_sup = soup.find_all("lo", {"class":'ui-search-layout ui-search-layout--grid'})
    divs = soup.find_all(main_div[0], {"class":main_div[1]})
    # print(divs)
    carros = []
    for item in divs:
        dict_list = {}
        clase_alta = BeautifulSoup(str(item),'html.parser')
        nombre_carro = clase_alta.find(estructura_1[0], itemprop=estructura_1[1]).text
        precio_carro = clase_alta.find(estructura_2[0], class_=estructura_2[1]).text
        imagenes = clase_alta.find_all(img_src[0], itemprop_=img_src[1])
        url_imagen = ''
        for img in imagenes:
            url_imagen = img.get('src')
            if not url_imagen.endswith('.webp'):
                url_imagen = img.get('data-src')
            else:
                url_imagen = img.get('src')
        dict_list['Nombre'] = nombre_carro
        dict_list['Precio'] = precio_carro
        dict_list['Imagen_URL'] = url_imagen
        carros.append(dict_list)
    return carros

In [54]:
url = 'https://www.autocosmos.com.co/catalogo/vigente/audi'
main_div = ['section','section m-brand']
estructura_1 = ['meta','model']
estructura_2 = ['span','model-card__price']
img_src = ['img','image']
carros_brand = extract_cars_2(url,main_div,estructura_1,estructura_2,img_src)

In [55]:
carros_brand

[{'Nombre': '',
  'Precio': '\n\n\n\nPrecio de lista a partir de:$155.400.000\n',
  'Imagen_URL': ''}]

In [91]:
import hashlib
def generate_image_name(img_url):
    # Utiliza un hash de la URL para generar un nombre único
    img_hash = hashlib.md5(img_url.encode()).hexdigest()
    return f'{img_hash}.jpg'

In [145]:
url = 'https://www.google.com/search?sca_esv=116b66c83e832583&hl=es&sxsrf=ADLYWIJnmkObXxD8Q-QL3DZe8bvgvJ2QYg:1728320665401&q=accidentes+carro&udm=2&fbs=AEQNm0CbCVgAZ5mWEJDg6aoPVcBgWizR0-0aFOH11Sb5tlNhdzvguW7TJ8ZJj4v-NOGupFiwZ48Dx3g7oaON1i1EOWFr4Xc7Fgj5H7_b7U_nkofi9rKU-iWnq3akONsyXiC15fc_IBFSshmfGsrcUEcLgDoqs8pLQOIVXv7pyxZ-bFASagYhN_qTShtUJbLXiufJYrtd4AGCg4WlSComD-Fvf5_swv72GA&sa=X&ved=2ahUKEwirvcaa4PyIAxVMfzABHc6lEXkQtKgLegQIFBAB&biw=1592&bih=791&dpr=1.2'

In [146]:
import requests
from bs4 import BeautifulSoup
import os
import re

save_directory = '/home/santenana/Proyectos/02_ObjectDetection/Accidentes_carros'

if not os.path.exists(save_directory):
    os.makedirs(save_directory)

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
images = soup.find_all('img')

def get_original_image_url(thumbnail_url):
    return thumbnail_url.replace('/tbn:', '/')  

for img in images:
    img_url = img.get('src') or img.get('data-src')
    if img_url:
        original_url = get_original_image_url(img_url)  
        try:
            img_data = requests.get(original_url).content
            img_name = os.path.join(save_directory, os.path.basename(original_url))
            with open(img_name, 'wb') as f:
                f.write(img_data)
        except Exception as e:
            continue


In [10]:
import os
import requests 
from bs4 import BeautifulSoup 
Google_Image = 'https://www.google.com/?hl=es'

u_agnt = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive',}

Image_Folder = '/home/santenana/Proyectos/02_ObjectDetection/Accidentes_carros'

def main():
    if not os.path.exists(Image_Folder):
        os.mkdir(Image_Folder)
    download_images()

def download_images():
    data = input('Enter your search keyword: ')
    num_images = int(input('Enter the number of images you want: '))
    print('Searching Images....')
    search_url = Google_Image + 'q=' + data 
    response = requests.get(search_url, headers=u_agnt)
    html = response.text 
    b_soup = BeautifulSoup(html, 'html.parser') 
    results = b_soup.findAll('img', {'class': 'rg_i Q4LuWd'})
    count = 0
    imagelinks= []
    for res in results:
        try:
            link = res['data-src']
            imagelinks.append(link)
            count = count + 1
            if (count >= num_images):
                break   
        except KeyError:
            continue
    print(f'Found {len(imagelinks)} images')
    print('Start downloading...')

    for i, imagelink in enumerate(imagelinks):
        response = requests.get(imagelink)
        imagename = Image_Folder + '/' + data + str(i+1) + '.jpg'
        with open(imagename, 'wb') as file:
            file.write(response.content)

    print('Download Completed!')
    

if __name__ == '__main__':
    main()
## Scraping process credits to: https://github.com/goutamborthakur555

Searching Images....
Found 0 images
Start downloading...
Download Completed!
